### Time Series Workshop 
# 5. Online Retail &#x1F4B0;: Forecasting Challenge

Now it's your turn! 

Here, we provide some guard rails to get you started. Try to
- explore the data
- Apply previously discussed techniques for pre-processing and feature engineering
- Forecast the next months sales for a pre-defined test set
- Evaluate the quality of your forecast

In [ ]:
%config InlineBackend.figure_format='retina'
%load_ext autoreload
%autoreload 2

from pathlib import Path

import matplotlib.pyplot as plt

from timeseries.data import load_retail

DATA_DIR = Path("..") / Path("data")

## Load data

In [ ]:
TARGET_COL = "sales"
SPLIT_DATE = "2011-09-30"
FILE_PATH = DATA_DIR / "online_retail.csv"

df_in = load_retail(FILE_PATH)
df_in.head()

## Analyze data
- Make yourselves familiar with the data. Here be dragons: There's a pitfall we haven't discussed yet, harhar. &#x1F60F;
- Do we have missing data?
- Can we see some obvious seasonal pattern? If so, what could be the reason for this?
- What else is peculiar about this data set?

In [ ]:
...

- How to deal with the trend? -> Seasonal decomposition!

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(..., model="additive")
fig = result.plot()
_ = fig.set_size_inches(15, 12)
_ = plt.tight_layout()

## Feature engineering
- Create some features that you think might be useful for forecasting
- Do we need to do some more pre-processing?

In [ ]:
...

## Train-test split
- Split the data into train- and test sets according to the SPLIT_DATE parameter defined above


In [ ]:
...

## Build models and forecast!
- Fit the processed training data 
- Predict for the test set
- Calculate the usual metrics
  - How good is your forecast? Compare a naive baseline model with something more sophisticated.
  - What metric is the most appropriate here?
  - Can you manage to beat my own forecast? (We will have a little competition here) &#x1F6A8;

Good luck!

In [ ]:
...